In [28]:
import datetime
import logging
import pickle
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from absl import logging as aLogging
aLogging.set_verbosity(aLogging.ERROR)
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
tf.print(tf. __version__)
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from collections import namedtuple
import glob

from keras.models import Sequential
from keras.layers import LSTM, Dense
import math
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.get_logger().setLevel(tf.compat.v1.logging.FATAL)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
logging.getLogger('tensorflow').setLevel(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')
# %pdb on

2.10.0


In [29]:
?tf

In [30]:
Stock = namedtuple('Stock', ['name', 'owned', 'dataframe', 'model'])

In [31]:
list_of_files = glob.glob('c:\\Users\\camar\\Downloads\\CAP5768\\data\\raw\\*.csv')

In [32]:
class Callback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        return False
    def on_epoch_end(self, epoch, logs=None):
        return False
    def on_batch_begin(self, batch, logs=None):
        return False
    def on_batch_end(self, batch, logs=None):
        return False
    def on_predict_batch_end(self, batch, logs=None):
        return False
    def on_predict_batch_begin(self, batch, logs=None):
        return False
    def on_train_begin(self, batch, logs=None):
        return False
    def on_train_end(self, batch, logs=None):
        return False
    def on_train_batch_begin(self, batch, logs=None):
        return False
    def on_train_batch_end(self, batch, logs=None):
        return False
    def on_test_begin(self, batch, logs=None):
        return False
    def on_test_end(self, batch, logs=None):
        return False


In [33]:
look_back = 1208
stocks = [None,None,None,None,None,None,None,None,None,None]

In [34]:
def predict(num_prediction, model, close_data):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list

In [38]:
def printTransactions(m, k, d, stocks, dayIndex):
    print('dayIndex', dayIndex)
#     print('k',k)
#     print('d',d)
    deltab = 0
    deltabi = -1
    # deltas = 999999999999999999
    # deltasi = -1
    lessThanZero = []
   
#     model = Sequential()
#     model.add(
#         LSTM(10,
#             activation='relu',
#             input_shape=(look_back,1))
#     )
#     model.add(
#         LSTM(10,
#             activation='relu',
#             input_shape=(look_back,1))
#     )
#     model.add(Dense(1))
#     model.compile(optimizer='adam', loss='mse')

    num_epochs = 25

    for i in range(k):
        stock = stocks[i]
        print('stock.model is None', stock.model is None, 'i', i, type(stock.model))
        if stock.model is None:
            model = Sequential()
            model.add(LSTM(units=50,return_sequences=True,activation='relu',input_shape=(look_back, 1)))
            model.add(Dropout(0.2))
            model.add(LSTM(units=100,return_sequences=True,activation='relu'))
            model.add(Dropout(0.2))
            model.add(LSTM(units=200,return_sequences=True,activation='relu'))
            model.add(Dropout(0.2))
            model.add(LSTM(units=100,activation='relu'))
            model.add(Dropout(0.2))
        #     model.add(LSTM(units=50,activation='relu'))
        #     model.add(Dropout(0.2))
            model.add(Dense(units=1))
            model.compile(optimizer='adam',loss='mean_squared_error') 
        else:
            model = stock.model
            
        print('name', stock.name)
        print('owned', stock.owned)
#         eprint('prices', stock.prices)
        df = stock.dataframe
        close_data = df['Close'].values
#         print(f"close_data.shape {close_data.shape}")
        close_data = close_data.reshape((-1,1))        
        train_generator = TimeseriesGenerator(close_data, close_data, length=min(look_back, close_data.shape[0] - 1),
                                              batch_size=20)
        model.fit(train_generator, epochs=num_epochs,callbacks=[Callback()], verbose=0)
#         ?model        
#         saver = model.fit.Saver()
#         print('type(saver)',type(saver))
        stock  = Stock(stock.name, stock.owned, stock.dataframe, model)
        stocks[i] = stock        
        pred1 = predict(1, model, close_data)[1]
        d = (pred1 - stock.dataframe.iloc[-1].Close) / stock.dataframe.iloc[-1].Close
#         print(f"pred1 {pred1}  d {d}   stock.dataframe.tail(1).iloc[0].Close {stock.dataframe.tail(1).iloc[0].Close}")
        if d > deltab and d > 0:
            deltab = d
            deltabi = i
        # if d < deltas and stock.owned > 0 and d < 0:
        #     deltas = d
        #     deltasi = i
        if d <= 0 and stock.owned > 0:
            lessThanZero.append(stock)
#     print(f"deltab {deltab}  deltabi {deltabi}")
            
    # if deltasi == deltabi:
    #     deltabi = -1 # sell
            
            
    toSpend = round(m * min(1, 0.8 + deltab), 2)  
    sb = stocks[deltabi]
    breakpoint()
    sharesb = math.floor(toSpend / sb.dataframe.iloc[dayIndex].Close)
    count = 1 if deltabi >= 0 and sharesb > 0 else 0
#     print(f" toSpend: {toSpend} sharseb {sharesb}")
    count += len(lessThanZero)
#     print(count)
    # eprint(count)
    if deltabi >= 0 and sharesb > 0:
        stocks[deltabi] = Stock(sb.name, sb.owned + sharesb, sb.dataframe, sb.model)
        m -= sb.dataframe.iloc[-1].Close * sharesb
        # eprint(f"{sb.name} BUY {sharesb}")    
    for sIndex in range(len(lessThanZero)):
        ss = lessThanZero[sIndex]
#         print(f"ss: {ss}  ss.dataframe.shape {ss.dataframe.shape}")
        m += ss.owned * ss.dataframe.iloc[-1].Close
        stockIndex = [i for i,stock in enumerate(stocks) if stock.name == ss.name][0]
        stocks[stockIndex] = Stock(ss.name, 0, ss.dataframe, ss.model)
    return m


In [39]:
def runDay(data, dayIndex):
#     print(f"runDay start data[1].[2].shape {data[1][2].shape}")
#     lstocks = []
#     try:
#         with open('stocks.pkl', 'rb') as f:
#             stocks = pickle.load(f)
#     except:
#         pass
#         # for line in lines.split('\n'):
#         #     # eprint(f'line |{line}|')
#         #     if len(line) > 4:
#         #         sa = line.split()
#         #         # eprint(f'line sa {sa}')
#         #         stocks.append(Stock(sa[0], 0, list(map(float, sa[1:]))))

    # eprint(len(stocks))
    # eprint(stocks[0])

    m,k,d = list(map(float, data[0].split()))
    k = int(k)
    d = int(d)
    for di in range(k):
        sa = data[di+1]
        dataframe = sa[2]
#         print(f"runDay dataframe.shape {dataframe.shape}")
#         print('stock[di].model is None before', stocks[di].model is None, 'di', di, type(stocks[di].model))
        stocks[di] = Stock(sa[0], int(sa[1]), dataframe, stocks[di].model)
#         print('stock[di].model is None after', stocks[di].model is None, 'di', di, type(stocks[di].model))
    print('printTransactions')
    m = printTransactions(m, k, d, stocks, dayIndex)
#     with open('stocks.pkl', 'wb') as f:
#         pickle.dump(stocks, f)
    return (m, stocks)

In [40]:
stockPrices = []
data = []
m = 10 # amount to spend each day
k = 10 # number of stocks
d = 0 # not used
data = []
data.append(f"{m} {k} {d}")
for i, csv in enumerate(list_of_files):
    name = os.path.basename(csv)[0:-4]
    df = pd.read_csv(csv)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_axis(df['Date'], copy=False)
    df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
    #,Volume,Open,High,Low
    df.rename(columns={"Close/Last": "Close", "Date": "Date"}, inplace=True) 
    df['Close'] = pd.to_numeric(df['Close'].map(lambda x: x[1:])) / 1000
#     df.info()
    assert df.loc[df['Close'].idxmax()].Close < 1
    data.append((name,0,df.iloc[0:look_back + 1]))
    stock = Stock(name, 0, df, None)
#     print('stock.model is None', stock.model is None, 'i', i)
    stocks[i] = stock
    stockPrices.append(stock)
print('runDay')    
m, stocks = runDay(data, look_back)
data.clear()
total = m + sum([k.owned * k.dataframe.iloc[look_back].Close for k in stocks])
print(f"total: {total}  m: {m}") 
with open('stocks.pkl', 'wb') as f:
    pickle.dump(stocks, f)
with open('stockPrices.pkl', 'wb') as f:
    pickle.dump(stockPrices, f)    
with open('m.pkl', 'wb') as f:
    pickle.dump(m, f)        

runDay
printTransactions
dayIndex 1208
stock.model is None True i 0 <class 'NoneType'>
name aapl
owned 0
1/1 [==============================] - 1s 1s/step
stock.model is None True i 1 <class 'NoneType'>
name amzn
owned 0
1/1 [==============================] - 2s 2s/step
stock.model is None True i 2 <class 'NoneType'>
name brk.b
owned 0
1/1 [==============================] - 2s 2s/step
stock.model is None True i 3 <class 'NoneType'>
name goog
owned 0
1/1 [==============================] - 1s 1s/step
stock.model is None True i 4 <class 'NoneType'>
name googl
owned 0
1/1 [==============================] - 1s 1s/step
stock.model is None True i 5 <class 'NoneType'>
name meta
owned 0
1/1 [==============================] - 1s 1s/step
stock.model is None True i 6 <class 'NoneType'>
name msft
owned 0
1/1 [==============================] - 2s 2s/step
stock.model is None True i 7 <class 'NoneType'>
name nvda
owned 0
1/1 [==============================] - 2s 2s/step
stock.model is None True i 8 <c

In [ ]:
stock = stockPrices[0]
stock.dataframe.loc[0:5].Close
stock.dataframe.iloc[-look_back:]
stock.dataframe.tail(1).iloc[0].Close
print(stock.dataframe.tail(1).iloc[0].Close)
print(stock.dataframe.iloc[-1].Close)


In [ ]:
with open('stocks.pkl', 'rb') as f:
    stocks = pickle.load(f)
with open('stockPrices.pkl', 'rb') as f:
    stockPrices = pickle.load(f)
with open('m.pkl', 'rb') as f:
    m = pickle.load(f)
data = []
k = 10
d = 0

In [41]:
def checkForNegative(stocks):
    if len([k for k in stocks if k.owned * k.dataframe.iloc[-1].Close < 0]) > 0:
        print(f"Negative Stock! {[(k, k.owned, k.dataframe.iloc[-1]) for k in stocks if k.owned * k.dataframe.iloc[-1].Close < 0]}")

In [ ]:
startTime = datetime.datetime.now()
for dayIndex in range(look_back + 1, max(stockPrices, key=lambda k: k.dataframe.shape[0]).dataframe.shape[0]):        
    data.append(f"{m} {k} {d}")
    for stock in stocks:
        dataframe = [s.dataframe for s in stockPrices if s.name == stock.name][0]
#         print(f"dataframe.iloc[0:dayIndex].shape {dataframe.iloc[0:dayIndex].shape}")
        if dataframe.shape[0] > dayIndex + 2:
            data.append((stock.name, stock.owned, dataframe.iloc[dayIndex - look_back:dayIndex + 2]))
        else:
#             data.append((stock.name, stock.owned, dataframe.iloc[-(look_back + 1):]))
            continue
  
    m, stocks = runDay(data, look_back)
    data.clear()
    checkForNegative(stocks)
#     if dayIndex % 20 == 0:
    total = m + sum([k.owned * k.dataframe.iloc[-1].Close for k in stocks])
    print(f"total: {total}  m: {m}  dayIndex: {dayIndex}   {datetime.datetime.now()}  {(datetime.datetime.now() - startTime).seconds} seconds")  
        
total = m + sum([k.owned * k.dataframe.iloc[-1].Close for k in stocks])
print(f"total: {total}  m: {m}  {datetime.datetime.now()}  {(datetime.datetime.now() - startTime).seconds} seconds")  



printTransactions
dayIndex 1208
stock.model is None False i 0 <class 'keras.engine.sequential.Sequential'>
name aapl
owned 0
1/1 [==============================] - 2s 2s/step
stock.model is None False i 1 <class 'keras.engine.sequential.Sequential'>
name amzn
owned 0
1/1 [==============================] - 1s 1s/step
stock.model is None False i 2 <class 'keras.engine.sequential.Sequential'>
name brk.b
owned 0
1/1 [==============================] - 1s 1s/step
stock.model is None False i 3 <class 'keras.engine.sequential.Sequential'>
name goog
owned 197
1/1 [==============================] - 1s 1s/step
stock.model is None False i 4 <class 'keras.engine.sequential.Sequential'>
name googl
owned 0
1/1 [==============================] - 2s 2s/step
stock.model is None False i 5 <class 'keras.engine.sequential.Sequential'>
name meta
owned 0
1/1 [==============================] - 2s 2s/step
stock.model is None False i 6 <class 'keras.engine.sequential.Sequential'>
name msft
owned 0
1/1 [========

In [ ]:
stockPrices[0].dataframe.shape

In [ ]:
%%debug